<a href="https://colab.research.google.com/github/GarlandZhang/gans_in_action_notes/blob/master/progessive_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras as K

Using TensorFlow backend.


In [2]:
def upscale_layer(layer, upscale_factor):
  height = layer.get_shape()[1]
  width = layer.get_shape()[2]
  size = (upscale_factor * height, upscale_factor * width)
  upscaled_layer = tf.image.resize_nearest_neighbor(layer, size)
  return upscaled_layer

In [3]:
def smoothly_merge_last_layer(list_of_layers, alpha):
  last_fully_trained_layer = list_of_layers[-2]
  last_layer_upscaled = upscale_layer(last_fully_trained_layer, 2)
  larger_native_layer = list_of_layers[-1]
  assert larger_native_layer.get_shape() == last_layer_upscaled.get_shape()
  new_layer = (1 - alpha) * upscaled_layer + alpha * larger_native_layer # alpha slowly increased as the larger native layer is being improved
  return new_layer

In [6]:
def minibatch_std_layer(layer, group_size=4): # group size is mini batch size
  group_size = K.backend.minimum(group_size, tf.shape(layer)[0])
  shape = list(K.int_shape(input))
  shape[0] = tf.shape(input)[0]
  minibatch = K.backend.reshape(layer, (group_size, -1, shape[1], shape[2], shape[3]))
  minibatch -= tf.reduce_mean(minibatch, axis=0, keepdims=True) # center the mean over the group
  minibatch = tf.reduce_mean(K.backend.square(minibatch), axis=0) # calculate variance
  minibatch = K.backend.square(minibatch + 1e8) # calculate standard dev
  minibatch = tf.reduce_mean(minibatch, axis=[1, 2, 4], keepdims=True) # takes average over all pixels
  minibatch = K.backend.tile(minibatch, [group_size, 1, shape[2], shape[3]]) # transform scalar value to fit groups and pixels (?)

  return K.backend.concatenate([layer, minibatch], axis=1) # appends as new feature map

In [7]:
def equalize_learning_rate(shape, gain, fan_in=None):
  '''
  This adjusts weights of every layer by the constant from He's initializer so that we adjust for the variance in the dynamic range in different features
  shape: shape of tensor (layer) or the dimensions
  e.g.; [4, 4, 48, 3] => [kernel_size, kernel_size, number_of_filters, feature_maps]
  '''

  if fan_in == None:
    fan_in = np.prod(shape[:-1]) # "the product of all the shape dimensions minus the feature maps dimension; this gives us the number of incoming connections per neuron"

  std = gain / K.sqrt(fan_in) # He's constant

  wscale = K.constant(std, name='wscale', dtype=np.float32) # "creates a constant out of the adjustment"

  adjusted_weights = K.get_value('layer', shape=shape, initializer=tf.initializers.random_normal()) * wscale # "Gets values for weights and then uses broadcasting to apply the adjustment"
  
  return adjusted_weights


In [8]:
def pixelwise_feat_norm(inputs, **kwargs):
  '''
  Use pixelwise feature normalization
  '''
  normalization_constant = K.backend.sqrt(K.backend.mean(inputs**2, axis=-1, keepdims=True) + 1.0e-8)

  return inputs / normalization_constant